### Objective: Convert all Unsearchable PDFS into Searchable PDFS. This includes cleaning, fixing / repairing , and image -preprocessing of unsearchable pdfs. These PDFS are then OCR'ed using Tesseract 4.0 with LSTM engines to get desired output pdf that is searchable

#### NOTE: output pdfs are "Sandwichs" meaning they contain the original pdf image overlayed on the OCR'ed text. HOCR output from teseract is used to re-align the extracted text over the image and maintain layout and position of text within the new pdf. 

#### OCRMYPDF Handbook: https://media.readthedocs.org/pdf/ocrmypdf/latest/ocrmypdf.pdf

In [8]:
#Import Modules
import os 
import re 


### 1.Identify Unserachable PDFS to Be Converted to Searchable 

In [9]:
#location of all POs , Results, and Cropped POs
path = "//home//kgplatformuser//python//ocr//samples//PDFs//"
outpath =  "//home//kgplatformuser//python//ocr//samples//SearchablePDF_Sandwiches//"

#identify searchable vs non searchable pdfs 
#note: pdfs must be stored in a directory with no other folders 
searchable , unsearchable = IsPDFSearchable(path)

#create paths to searchable pdfs (only pull pack the ones that have been cropped and extra pages removed)
UnSearchablePDFPaths = [path + fn for fn in unsearchable if fn.endswith('.pdf')]     
print(UnSearchablePDFPaths)


['//home//kgplatformuser//python//ocr//samples//PDFs//PO1.pdf', '//home//kgplatformuser//python//ocr//samples//PDFs//PO2.pdf', '//home//kgplatformuser//python//ocr//samples//PDFs//PO10.pdf', '//home//kgplatformuser//python//ocr//samples//PDFs//PO13_Email.pdf', '//home//kgplatformuser//python//ocr//samples//PDFs//PO7.pdf']


### 2. Use OCRMYPDF To repair, pre-process and configure original pdf into searchable pdf using tesseract 

In [10]:
#generate pdf-a (default output for ocrmypdf)
SearchablePDFOutpaths= []
for i in range(len(UnSearchablePDFPaths)):
    pdf_output_path = outpath + str(UnSearchablePDFPaths[i].rsplit("//", 1)[1]).replace(".pdf","")+"_V2searchable.pdf"
    #os.system("ocrmypdf --output-type pdf --deskew --clean --rotate-pages --clean-final "+ UnSearchablePDFPaths[i]+" "+ pdf_output_path )
    SearchablePDFOutpaths.append(pdf_output_path)          
    print(pdf_output_path)

//home//kgplatformuser//python//ocr//samples//SearchablePDF_Sandwiches//PO1_V2searchable.pdf
//home//kgplatformuser//python//ocr//samples//SearchablePDF_Sandwiches//PO2_V2searchable.pdf
//home//kgplatformuser//python//ocr//samples//SearchablePDF_Sandwiches//PO10_V2searchable.pdf
//home//kgplatformuser//python//ocr//samples//SearchablePDF_Sandwiches//PO13_Email_V2searchable.pdf
//home//kgplatformuser//python//ocr//samples//SearchablePDF_Sandwiches//PO7_V2searchable.pdf


## 3. Use PDF2Text to Generate Layout specific text file associated with Each 

In [11]:
#return text(can be used to scan for key header information)
from collections import OrderedDict
from operator import itemgetter
from StringIO import StringIO
import stat
import os
import sys, getopt
from subprocess import Popen, PIPE

path = "//home//kgplatformuser//python//ocr//samples//SearchablePDF_Sandwiches//"
files = getfiles("//home//kgplatformuser//python//ocr//samples//SearchablePDF_Sandwiches//", ".pdf")
pdftextpaths=[]
for i in range(len(files)):
    #txtpath = SearchableCropPDFPaths[i]+str(SearchableCropPDFPaths[i].rsplit("//", 1)[1]).replace("_CroppedExPgsRmvd.pdf",".txt")
    txtpath = path + files[i].replace("_V2searchable.pdf",".txt")
    pdfpath = path + files[i]
    pdftextpaths.append(get_text_file(pdfpath, txtpath))

## 4.(Option 1) Load Text into Pandas DF  and Parse Attributes

In [78]:
import re
import pandas as pd
from re import finditer

#Open file and split lines into list
f =open(pdftextpaths[0])
a = f.read()
b = a.split("\n")

#remove all completely white lines (helps postprocessing and identifying values that are below associated attribute)
c = [x for x in b if x]
#print(c)

#increase max column width to display
pd.options.display.max_colwidth = 300

#attribute tuple (holds all attributes and the regex used to find each)
LookupTable = zip(['PO NUMBER'],[re.compile(r'P.O. NO.:|P(.)?(/)?O(.)?(#)? | Purchase Order (#)?|PURCHASE ORDER(#)?')])
#print(LookupTable)

#NOTE: TEST contains all matches for the element in question
#identify start and end locaiton of string found for each line in po
test =[]
for i in range(len(c)):
    
    #for each pattern match found in line i
    for match in finditer(LookupTable[0][1], c[i]):
    
        #output nested list with tuple of locational information about the match 
        test.append(zip([str(i)],[match.span()], [match.group()])) #create a 3 element tuple with the information on locaiton of string

#print(test)
#split the nested tuples to extract key elements 
#syntax:#test[0]:first list element ,test[0][0]:first element in nested list(the tuple),test[0][0][2]:third element in tuple 
#print(test[0][0][2]) 

#Desriptive sttatistics about line in which matches were found 
matchdetails = []
for i in range(len(test)):
    
    #Aggregated details on matched strings  
    matchdetails.append(zip( 
                             [int(test[i][0][0])]                             #Line in text inwhich the match was found 
                           , [wordCount(stripwhite(c[int(test[i][0][0])]))]   #Totalwords in line string was found
                           , [len(c[int(test[i][0][0])])]                     #Length of characters in the line the match was found 
                           , [test[i][0][1]]                                  #Start and end position of the matched pattern found 
                           , [test[i][0][2]]                                  #Actual value the pattern matched on 
                           ))
    

#processing each match to find true result
    if LookupTable[0][0] == 'PO NUMBER':
        #string being processed
        string = c[int(test[i][0][0])]
        #print(string)
        
        #all text to the right of the string matched on
        ToRight_Orig = string[string.find(test[i][0][2])+len(test[i][0][2]):len(string)] #starting position of match + num characters associated with attribute being searched for : eol
        ToRight_Strip = string[string.find(test[i][0][2])+len(test[i][0][2]):len(string)].strip()

        if len(ToRight_Strip)> 0 and len(ToRight_Strip.split()[0])>len(test[i][0][2]): #ensures no null values are passed and inccorect word is passed 
        #next word in string after the matched word 
            nextword = ToRight_Strip.split()[0]
            #print(nextword)
            #if next word contains at least 1 letter and one digit return and can contain a special character (-)
            for match in finditer(re.compile(r'^(?=.*[\d].*)[a-zA-Z\d!#-*]{5,}$'), nextword):
                print(match.group())


    
    


[('PO NUMBER', <_sre.SRE_Pattern object at 0x5671620>)]
PP6179676


In [ ]:

    #string being processed
    string = c[int(test[i][0][0])]
   
    #all text to the right of the string matched on
    ToRight = string[string.find(test[i][0][2])+len(test[i][0][2]):len(string)] 
    #next word in string after the matched word 
    nextword = ToRight.split()[0]
    
    #all text below string matched on 
    BelowString = c[int(test[i][0][0])+1]
    LenBelowString = len(c[int(test[i][0][0])+1]) #length of string on row i+1
  
    BelowStartPos = test[i][0][1][0] #if we are pulling information from string below, we want to start around where text is aligned to header
     
    #if info is below text, we must determine where to start pulling from in relation to the header
    FindManySpacesFromEnd = BelowString.find("   ",LenBelowString/2,-10) #returns the first occurance of the "multiple spaces" found in string 
    FindManySpacesFromStart = BelowString.find("   ",0,LenBelowString) #returns the first occurance of the "multiple spaces" found in string 
    FindManySpacesFromheader = BelowString.find("   ",BelowStartPos,LenBelowString) #returns the first occurance of the "multiple spaces" found in string 
    
    
    BelowAreaofInterst = BelowString[BelowStartPos:len(BelowString)]
    #print(FindManySpaces,LenBelowString,BelowString)


#list_of_words = b[int(test[i][0][0])].split() #split string into list of words 
#print(list_of_words)
#next_word = list_of_words[list_of_words.index(test[i][0][2])+1]
    

In [14]:
#print(test[1][0][0]) 


wordCount(countwords(b[5]))

NameError: name 'countwords' is not defined

## 4.(Option 2) Load Text into list  and Parse Attributes

In [77]:
#read in formatted text file and look at content

import pandas as pd
import io
import os 
import re 
import codecs
import validate_email 
#import PYDNS #we can actually validate the email addresses are correct and active if we can get this to run

pd.options.display.max_colwidth = 300

#Datasets for testing 
path = "//home//kgplatformuser//python//ocr//samples//SearchablePDF_Sandwiches//"
files = getfiles("//home//kgplatformuser//python//ocr//samples//SearchablePDF_Sandwiches//", ".txt")
#print(files)

#note: must figure out how to identify total characters required to get to next line , below the position of the attribute feild name
in_file = open(path + files[3],"rt") # open file lorem.txt for reading text data
contents = in_file.read()         # read the entire file into a string variable
in_file.close()                   # close the file
in_file
print(len(contents), type(contents))
print(contents)# print contents
print(type(contents))



(2056, <type 'str'>)
CBPacific©                                                                    Purchase Order




                                                                             Date                       6/29/2017
CB Pacific, Inc
                                                                             PO #                       PP6179676
Send invoice to:
Attn: Accounts Payable
909 Seventh Ave
Suite 201
Kirkland WA 98033
Ph. 5425; B22—1702
Fx.   (425)   822—5442




Vendor                                Vendor Fax               Shipping Met...          Ship Date           Carrier Pay
GE Intelligent Platforms, Inc. [...
                                                               See Text                 £/29/2017           PPD/Charge
2500 Austin Drive
Charlottesville VA 22911              Rep Code                  Buyer                   Terms
United States
                                                                Lauri J Lerner          Net
               

In [71]:
#parse the key terms

#https://www.pythonlearn.com/html-008/cfbook007.html
atpos = []
atpos2 = []
with open(path + files[0],"rt") as in_file:  # Open file lorem.txt for reading of text data.
    i=0

    #first loop through each line and determine if keyword is present in line
    for line in in_file:        
        data = line 
        atpos.append([i,data.find('fax')])
        i = i+1
            
    #second, subset returned list to only include rows(i) where the keyword was found 
    for i in range(len(atpos)): 
        if atpos[i][1] != -1:
            atpos2.append(atpos[i])
    
    #third: using the starting position of the keyword, look for the next occurance of 2 spaces and pull back whats to the right
    for i in range(len(atpos2)):
        loc = atpos2[i][2].split('P.O. NO.:',1)[1]
        sppos = loc[0][2].find('   ',loc[0][1])
        #print(sppos)




[[0, -1],
 [1, -1],
 [2, -1],
 [3, -1],
 [4, -1],
 [5, -1],
 [6, -1],
 [7, -1],
 [8, -1],
 [9, -1],
 [10, -1],
 [11, -1],
 [12, -1],
 [13, -1],
 [14, -1],
 [15, -1],
 [16, -1],
 [17, -1],
 [18, -1],
 [19, -1],
 [20, -1],
 [21, -1],
 [22, -1],
 [23, -1],
 [24, -1],
 [25, -1],
 [26, -1],
 [27, -1],
 [28, -1],
 [29, -1],
 [30, -1],
 [31, -1],
 [32, -1],
 [33, -1],
 [34, -1],
 [35, -1],
 [36, -1],
 [37, -1],
 [38, -1],
 [39, -1],
 [40, -1],
 [41, -1],
 [42, -1],
 [43, -1],
 [44, -1],
 [45, -1],
 [46, -1],
 [47, -1],
 [48, -1],
 [49, -1],
 [50, -1],
 [51, -1]]

In [14]:
#create a list from the text file and search for terms (just for show reg-ex used in later steps)
import re
import pandas as pd

pd.options.display.max_colwidth = 300

f =open(pdftextpaths[0],"r")
a = f.read()

in_file = open(pdftextpaths[2],"rt") # open file lorem.txt for reading text data
contents = in_file.read()         # read the entire file into a string variable
in_file.close()                   # close the file
#print(contents)                 # print contents

lines = []
with open(pdftextpaths[2],"rt") as in_file:  # Open file lorem.txt for reading of text data.
    for line in in_file: # Store each line in a string variable "line"
        lines.append(line) #add line to list of lines
        #print(line) # prints that line
        
        
#find a sting in a line 
print(len(lines)) 
print(lines[0].find("Pacific")) #tells us the word Pacific starts at the second character in the line 
print(lines[0].find("Pacific",30)) #tells find to start searching for string beginigng at the 30th character of the string
print(lines[10])

50
-1
-1
DATE: Sept. 8, 2016                                P.O. NO.:         AHISC—2016—0809—04



In [37]:
#string functions in python that look useful
#word.isalnum()         #check if all char are alphanumeric 
#word.isalpha()         #check if all char in the string are alphabetic
#word.isdigit()         #test if string contains digits
#word.istitle()         #test if string contains title words
#word.isupper()         #test if string contains upper case
#word.islower()         #test if string contains lower case
#word.isspace()         #test if string contains spaces
#word.endswith('d')     #test if string endswith a d
#word.startswith('H')   #test if string startswith H


#Phone and Tax 
phoneprefix = re.compile(r'Tel(.)?(.*)|TEL(.)?(#)?(.*)|Telephone(.*)|telephone(.*)|phone( number)?(.*)')
phoneRegex = re.compile(r'''(
    (\d{3}|\(\d{3}\))?                # area code
    (\s|-|\.)?                        # separator
    (\d{3})                           # first 3 digits
    (\s|-|\.)                         # separator
    (\d{4})                           # last 4 digits
    (\s*(ext|x|ext.)\s*(\d{2,5}))?    # extension
    )''', re.VERBOSE)

faxprefix = re.compile(r'Fax(.*)')
phoneNumDigits = re.compile(r'((\d){3}(-)?(\d){3}(-)?(\d){4})')
phoneNumRegex2 = re.compile(r'(\d\d\d\d\d\d\d\d\d\d)')

#email 
email = re.compile(r'([^@|\s]+@[^@]+\.[^@|\s]+)')

#table start (identify first column in table and pull back entire line)
table1 = re.compile(r'Item #(.*)|Line(.*)|LINE(.*)|ITEM(.*)')

#PO NUMBER 
PONumRegex = re.compile(r'P(.)?(/)?O(.)?(#)? | Purchase Order (#)?')

#mo = phoneNumRegex.search('My number is 415-555-4242.')
#print('Phone number found: ' + mo.group())

#trying to loop through all lines in text to identify phone numbers
for i in range(len(lines)):
    inpt = re.sub('[\s]+', '', lines[i])
    
    phone = phoneRegex.findall(str(inpt))
    PONumber = PONumRegex.findall(lines[i])
    phonepre =phoneprefix.findall(lines[i], re.I) #case insensative
    faxpre =faxprefix.findall(lines[i], re.I) #case insensative
    tablestart = table1.findall(lines[i]) #leave case sensative to prevent pulling back random words 
    emails = email.findall(lines[i], re.I)
    

    #line.group()
    print(phonetest)
    
    #for j in range(len(line)):
    #    chunk = line[j:j+11]
        #try: 
        #    chunk2 =int(line[j:j+11])
        #except: 
        #    chunk2 = line[j:j+11]
        #print(chunk2)
    #if isPhoneNumber(chunk):
    #    print('Phone number found: ' + chunk)
    #print('Done')
    
#print(re.sub('[^A-Za-z0-9]+', '', lines[5]))   
#isPhoneNumber(re.sub('[^A-Za-z0-9]+', '', lines[5]) )


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('604.669.6674', '604', '.', '669', '.', '6674', '', '', ''), ('604.669.2902', '604', '.', '669', '.', '2902', '', '', '')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


list

[]

In [74]:
##PULLING BACK LOCATION OF THE MATCHED PATTERN

import re
import os # You can go without is if you have other means to get your filepath

i = 0
matches = []
target =table1

#identifies all matches and returns location 
with open(pdftextpaths[2],"rt") as fic: # open("myfile.txt") if in your current directory
    for line in fic:
        if re.search(target, line):
            #print "Found at line {}".format(i)
            matches.append(i)
        i = i +1
if not len(matches):
    raise Exception, "target not found"
    
#return table information 
print(lines[matches[0]])
lines[matches[0]-1:matches[0]+12]

#find total characters into new line each element occurs 
lines[matches[0]-1].find("#")


#look at information to left and right of the string found
#note: it may be helpful to remove all blank lines during this parsing step to prevent pulling back no informaiton
left_text = lines[matches[0]-1].partition("   ")[0]
right_text = lines[matches[0]-1].partition("   ")[2]

left_text_nextline = lines[matches[0]].partition("   ")[0]
right_text_nextline = lines[matches[0]].partition("   ")[2]

left_text_next2line = lines[matches[0]+6].partition("   ")[0]
right_text_next2line = lines[matches[0]+6].partition("   ")[2]

print(left_text,right_text)
print(left_text_nextline, right_text_nextline)
print(left_text_next2line, right_text_next2line)

#print(len(lines[1])) #max characters in line 
#start character in line that is not empty (not space)
#

Item #                                     PARTICULAR                                       Unit Cost       [Amount (in USD)

('\n', '')
('Item #', '                                  PARTICULAR                                       Unit Cost       [Amount (in USD)\n')
('', '      ~|Implementation as per the agreed Statement of Work (SoW)                        Lump Sum              195,000.00\n')


### 4. (Option 2) Generate xml output and parse 

In [7]:
#specify input/output paths
inputpdf = "//home//kgplatformuser//python//ocr//samples//SearchablePDF_Sandwiches//PO1_V2searchable.pdf"
outputxml = "//home//kgplatformuser//python//ocr//samples//xmlconverts//PO1.xml"

#run pdftohtml
#Note:It is important that you specifiy the -hidden parameter when you're dealing with OCR-processed ("sandwich") PDFs.
#You can furthermore add the parameters -f n and -l n to set only a range of pages to be converted.
#note: use subprocess going forward subprocess.call("exit 1", shell=True)
os.system("pdftohtml -c -hidden -xml -wbt 13 "+inputpdf+" "+outputxml)


NameError: name 'os' is not defined

In [ ]:
#Option 1: reading file in its entirty (parsing is soley baseed on character location, not line number)
#note: must figure out how to identify total characters required to get to next line , below the position of the attribute feild name
in_file = open(pdftextpaths[0],"rt") # open file lorem.txt for reading text data
contents = in_file.read()         # read the entire file into a string variable
in_file.close()                   # close the file
#print(len(contents), type(contents))
#print(contents[1:2113])                 # print contents

print(lines[20], end = '')
for i in range(21,27):
        print(lines[i])
#print(lines[21])
#print(lines[21])
#print(lines[21])


In [ ]:
#create lookup list and identify hits 


in_file = open(pdftextpaths[0],"rt") # open file lorem.txt for reading text data
contents = in_file.read()         # read the entire file into a string variable
in_file.close()                   # close the file
print(len(contents), type(contents))
print(contents[1:343])                 # print contents




In [ ]:
## 3.Convert all Sandwiched PDFS into text files using pdftotext
pdftextpaths =[]
for i in range(len(SearchablePDFOutpaths)):
    txtpath = SearchablePDFOutpaths[i].replace("_V2searchable.pdf","Searchable_text.txt")
    pdfpath = SearchablePDFOutpaths[i]
    pdftextpaths.append(get_text_file(pdfpath, txtpath))


In [ ]:


#return output from command line to variable 
import subprocess

proc = subprocess.Popen(["cat", "/etc/services"], stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()
print "program output:", out

## Functions 

In [15]:

##Function to determine if pdf is searchable 
def IsPDFSearchable(path): 
    
    #determine if pdf is searchable 
    import os, re
    
    #path is the directory with the files, other 2 are the names of the files you will store your lists in
    path = path
    
    #use this if u want to actually create a file holding all pdfs that are searchable vs not 
    #files_with_text = open("C://Users//caridza//Desktop//EY//AI//OCR//GE PO RFP//solution data//PO Examples//files_with_text.txt", 'a')
    #image_only_files = open("C://Users//caridza//Desktop//EY//AI//OCR//GE PO RFP//solution data//PO Examples//image_only_files.txt", 'a')
    
    files_with_text = []
    image_only_files = []
    
    #have os make a list of all files in that dir for a loop
    filelist = os.listdir(path)
    
    #compile regular expression that matches "Font"
    mysearch = re.compile(r'.*Font.*', re.DOTALL)
    
    #loop over all files in the directory, open them in binary ('rb'), search that binary for "Font"
    #if they have "Font" they have text, if not they don't
    #(pdf does something to understand the Font type and uses this word every time the pdf contains text)
    for pdf in filelist:
        openable_file = os.path.join(path, pdf)
        cat_file = open(openable_file, 'rb')
        usable_cat_file = cat_file.read()
        #print usable_cat_file
        if mysearch.match(usable_cat_file):
            #files_with_text.write(pdf + '\n') #use if u are storing info in file
            files_with_text.append(pdf)
        else:
            #image_only_files.write(pdf + '\n') #use if u are storing info in file 
            image_only_files.append(pdf)
    
    #use if storing pdf names in files
    #close files holding searchable vs unsearchable pdfs
    #files_with_text.close()
    #image_only_files.close()
    return files_with_text, image_only_files


##https://www.freebsd.org/cgi/man.cgi?query=pdftotext&apropos=0&sektion=0&manpath=FreeBSD+6.2-RELEASE+and+Ports&format=html
#http://www.pdftron.com/assets/pdfs/support/PDFTron_PDF2Text_User_Manual.pdf
#options not used 
##'-remove_hidden_text',
##'-remove_invisible_text'
## pdf2text -c 150,600,250,700 license.pdf -a 1 #extract text from a speicific region 
def get_bbox_file(pdf_path, txt_path):
    if not os.path.exists(pdf_path):
        raise FileNotFoundError("Couldn't find {}".format(pdf_path))
    elif os.path.exists(txt_path):
        return txt_path
    # TXT file doesn't exist yet. Create it.
    try:
        p1 = Popen(('pdftotext','-bbox-layout',  
                       pdf_path, txt_path),
                      stdin=PIPE, stdout=PIPE)
        output = p1.communicate()
    finally:
        p1.stdout.close()
        p1.stdin.close()
    try:
        p1.terminate()
        p1.kill()
    except:
        pass

    return txt_path
    
#return tables 
#return bounding boxes
#for i in range(len(SearchablePDFPaths)):
#    bboxpath = croppath+str(SearchablePDFPaths[i].rsplit("//", 1)[1]).replace(".pdf","_BBox.txt")
#    pdfpath = SearchablePDFPaths[i]
#    bboxtextpaths.append(get_bbox_file(pdfpath, bboxpath))

#extract only bounding box information
#lines = []
#for i in range(len(bboxtextpaths)):
#    #lines.append([ line for line in open(bboxtextpaths[i]) if 'block' in line])
#    lines.append([ line.replace("</block>\n","").replace("<block","").replace(">\n","").replace(",","") for line in open(bboxtextpaths[i]) if 'block' in line])

def get_text_file(pdf_path, txt_path):
    if not os.path.exists(pdf_path):
        raise FileNotFoundError("Couldn't find {}".format(pdf_path))
    elif os.path.exists(txt_path):
        return txt_path
    # TXT file doesn't exist yet. Create it.
    try:
        p1 = Popen(('pdftotext','-layout',  
                       pdf_path, txt_path),
                      stdin=PIPE, st=PIPE)
        output = p1.communicate()
    finally:
        p1.stdout.close()
        p1.stdin.close()
    try:
        p1.terminate()
        p1.kill()
    except:
        pass

    return txt_path

#get files
def getfiles(path, extension):
    items = os.listdir(path)
    newlist = []
    for names in items:
        if names.endswith(extension):
            newlist.append(names)
    return newlist


def isPhoneNumber(text, chars = 11):
    if len(text) != chars:
        return False
    
    for i in range(0, 2):
        if not text[i].isdecimal():
            return False
        #if text[3] != '-':
        #    return False
    for i in range(3, 6):
        if not text[i].isdecimal():
            return False
        #if text[7] != '-':
        #    return False
    for i in range(chars,11):
        if not text[i].isdecimal():
            return False
    return text


def validate(email): 
    match=re.search(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9]+\.[a-zA-Z0-9.]*\.*[com|org|edu]{3}$)",email)
    if match:
        return 'Valid email.'
    else:
        return 'Invalid email.'
    

#count words in string 
def wordCount(mystring):  
    tempcount = 0  
    count = 1  
    indicator = 0  
    try:  
        for character in mystring:  
            if character == " ":  
                tempcount +=1  
                if tempcount ==1:  
                    count +=1  

                else:  
                    tempcount = 0  
        return count  

    except Exception:  
        error = "Not a string"  
        return error  

#remove all white space from string (only use to count total words in line)
def stripwhite(string):

    string = string.rstrip().lstrip() # Remove all extra spaces at the start and at the end of the string
    while "  " in string: # While  there are 2 spaces beetwen words in our string...
        string = string.replace("  ", " ") # ... replace them by one space!
    return(string)

